In [1]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector import connect,DictCursor
import pandas as pd
import requests
import json
from datetime import datetime

In [2]:
api_key = "6567132011mshed68849c12b0db0p10f15ejsnd58ad5ce4767"  # Aksh
# api_key = "4f16ab08e8msh27e320e3d194653p1e0308jsn539b154232c5" # Divy
# api_key = "8620937591msh4e8402b095c1122p1aa86cjsncc3f82db766b" # Ikram

In [3]:
headers = {
            "x-rapidapi-key": f"{api_key}",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }

http_url = "https://cricbuzz-cricket.p.rapidapi.com"

In [36]:
conn = snowflake.connector.connect(
 user='project',
 password='Project@123456',
 account='fzkreem-ojb05768',
 warehouse = 'COMPUTE_WH',
 role = 'ACCOUNTADMIN',
)

In [37]:
cs = conn.cursor()
# cs.execute('CREATE or REPLACE DATABASE "cricbuzz"')
cs.execute('USE DATABASE "cricbuzz"')

In [157]:
# 1. Fetch data from API
# match_id = 100283
# match_id = 100290
# match_id = 100292
# match_id = 100301
# match_id = 100310
# match_id = 100312
match_id = 100321
# match_id = 100328

def fetch_match_data():
    url = f"{http_url}/mcenter/v1/{match_id}/hscard"
    response = requests.get(url,headers=headers)
    if response.status_code == 200:
        return response.json()  # Convert response to JSON
    else:
        print("Failed to fetch data:", response.status_code)
        return None

In [ ]:
# create_series_table_query = """
# CREATE or replace TABLE Series (
#     seriesId number(10,0) PRIMARY KEY,
#     seriesName STRING,
#     seriesDesc STRING,
#     year INT
# )
# """
# cs.execute(create_series_table_query)

# create_match_table_query = """
# CREATE TABLE Matches (
#     matchId number(10,0) PRIMARY KEY,
#     seriesId number(10,0),
#     matchDescription STRING,
#     matchFormat STRING,
#     matchType STRING,
#     matchStartTimestamp TIMESTAMP,
#     matchCompleteTimestamp TIMESTAMP,
#     tossWinnerId number(10,0),
#     tossWinnerName STRING,
#     decision STRING,
#     winningTeam STRING,
#     winningteamId number(10,0),
#     winningMargin number(10,0),
#     winByRuns BOOLEAN,
#     winByInnings BOOLEAN,
#     status STRING,
#     FOREIGN KEY (seriesId) REFERENCES Series(seriesId)
# )
# """

# create_innigs_table_query = """
# CREATE TABLE Innings (
#     inningsId number(10,0),
#     matchId number(10,0),
#     battingTeamId number(10,0),
#     bowlingTeamId number(10,0),
#     totalRuns number(10,0),
#     totalWickets number(10,0),
#     overs number(10,2),
#     runRate number(10,2)
# );
# """


# create_BattingPerformance_table_query = """
# CREATE or replace TABLE BattingPerformance (
#     matchId number(10,0),
#     inningsId number(10,0),
#     batsmanId number(20,0),
#     batsmanName varchar(100),
#     runs number(10,0),
#     balls number(10,0),
#     fours number(10,0),
#     sixes number(10,0),
#     strikeRate number(10,2),
#     outDesc STRING,
#     bowlerId number(10,0),
#     fielderId1 number(10,0),
#     fielderId2 number(10,0),
#     wicketCode STRING
# )
# """

# create_BowlingPerformance_table_query = """
# CREATE TABLE BowlingPerformance (
#     matchId number(10,0),
#     inningsId number(10,0),
#     bowlerId number(10,0),
#     bowlerName varchar(100),
#     overs number(10,2),
#     maidens number(10,0),
#     runsConceded number(10,0),
#     wickets number(10,0),
#     economy number(10,2),
#     noBalls number(10,0),
#     wides number(10,0),
#     dots number(10,0)
# );
# """

# create_wicket_table_query = """
# CREATE TABLE Wickets (
#     matchId number(10,0),
#     inningsId number(10,0),
#     wktNbr number(10,0),
#     batId number(10,0),
#     batName varchar(100),
#     wktOver number(10,2),
#     wktRuns number(10,0)
# );
# """

create_partnership_table_query = """
CREATE TABLE Partnerships (
    matchId number(10,0),
    inningsId number(10,0),
    partnershipNbr number(10,0),
    bat1Id number(10,0),
    bat1Name varchar(100),
    bat1Runs number(10,0),
    bat1balls number(10,0),
    bat2Id number(10,0),
    bat2Name varchar(100),
    bat2Runs number(10,0),
    bat2balls number(10,0),
    totalRuns number(10,0),
    totalBalls number(10,0)
);
"""

cs.execute(create_partnership_table_query)

In [ ]:
# Insert Series Data

def insert_series(cursor, series_data):
    # cursor = conn.cursor()
    sql = """INSERT INTO Series (seriesId, seriesName, seriesDesc, year) VALUES (%s, %s, %s, %s)"""
    cursor.execute(sql, (series_data["seriesId"], series_data["seriesName"], series_data["seriesDesc"], series_data["year"]))
    # conn.commit()

In [21]:
def insert_match(cursor, match_data):
    # cursor = conn.cursor()
    sql = """INSERT INTO Matches (matchId, seriesId, matchDescription, matchFormat, matchType, matchStartTimestamp, matchCompleteTimestamp,
              tossWinnerId, tossWinnerName, decision, winningTeam, winningteamId, winningMargin, winByRuns, winByInnings, status)
              VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
        # Convert matchStartTimestamp and matchCompleteTimestamp to TIMESTAMP format
    match_start_timestamp = datetime.fromtimestamp(match_data["matchStartTimestamp"] / 1000)
    match_complete_timestamp = datetime.fromtimestamp(match_data["matchCompleteTimestamp"] / 1000)
    
    cursor.execute(sql, (
        match_data["matchId"], match_data["seriesId"], match_data["matchDescription"], match_data["matchFormat"],
        match_data["matchType"], match_start_timestamp, match_complete_timestamp,
        match_data["tossResults"]["tossWinnerId"], match_data["tossResults"]["tossWinnerName"],
        match_data["tossResults"]["decision"], match_data["result"]["winningTeam"], match_data["result"]["winningteamId"],
        match_data["result"]["winningMargin"], match_data["result"]["winByRuns"], match_data["result"]["winByInnings"],
        match_data["status"]
    ))
    # conn.commit()

In [ ]:
def insert_players(cursor, players_data, team_id):
    # i = 0
    # for player in players_data:
    #     url = f"{http_url}/stats/v1/player/{player["id"]}"
    #     response = requests.get(url,headers=headers)
    #     if response.status_code == 200:
    #         data =  response.json()
    #     if i == 0:
    #         break
    sql = "INSERT INTO ""team_players"" (id, name,battingStyle,bowlingStyle,role teamId) VALUES (%s, %s, %s, %s, %s,%s,%s)"
    
    for player in players_data:
        url = f"{http_url}/stats/v1/player/{player["id"]}"
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            data =  response.json()
        
        cursor.execute(sql, (
            player["id"], player["fullName"],data["bat"],data["bowl"],data["role"], team_id
        ))
    # conn.commit()

In [100]:
def insert_innings(cursor, innings_data):
    # cursor = conn.cursor()
    sql = """INSERT INTO Innings (inningsId, matchId, battingTeamId, bowlingTeamId, totalRuns, totalWickets, overs, runRate)
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""
    
    cursor.execute(sql, (
        innings_data["inningsId"], innings_data["matchId"], innings_data["batTeamDetails"]["batTeamId"],
        innings_data["bowlTeamDetails"]["bowlTeamId"], innings_data["scoreDetails"]["runs"],
        innings_data["scoreDetails"]["wickets"], innings_data["scoreDetails"]["overs"], innings_data["scoreDetails"]["runRate"]
    ))
    # conn.commit()

In [72]:
def insert_batting_performance(cursor, batting_data, matchId, inningsId):
    # cursor = conn.cursor()
    sql = """INSERT INTO BattingPerformance (matchId, inningsId, batsmanId, batsmanName, runs, balls, fours, sixes, strikeRate, outDesc, bowlerId, fielderId1, fielderId2, wicketCode) 
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s)"""
    
    for _, batsman in batting_data.items():
        print(batsman)
        cursor.execute(sql, (
            matchId,
            inningsId,
            batsman["batId"],
            batsman["batName"],
            batsman["runs"],
            batsman["balls"],
            batsman["fours"],
            batsman["sixes"],
            batsman["strikeRate"],
            batsman["outDesc"],
            batsman["bowlerId"],
            batsman["fielderId1"],
            batsman["fielderId2"],
            batsman["wicketCode"]
        ))
        print(batsman)
    # conn.commit()

In [80]:
def insert_bowling_performance(cursor, bowling_data, matchId, inningsId):
    # cursor = conn.cursor()
    sql = """INSERT INTO BowlingPerformance (matchId, inningsId, bowlerId,bowlerName, overs, maidens, runsConceded, wickets, economy, noBalls, wides, dots) 
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"""
    
    for _, bowler in bowling_data.items():
        cursor.execute(sql, (
            matchId,
            inningsId,
            bowler["bowlerId"],
            bowler["bowlName"],
            bowler["overs"],
            bowler["maidens"],
            bowler["runs"],
            bowler["wickets"],
            bowler["economy"],
            bowler["no_balls"],
            bowler["wides"],
            bowler["dots"]
        ))
    # conn.commit()

In [83]:
def insert_wickets(cursor, wickets_data, matchId, inningsId):
    # cursor = conn.cursor()
    sql = """INSERT INTO Wickets (matchId, inningsId, wktNbr, batId, batName, wktOver, wktRuns)
             VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    
    for _, wicket in wickets_data.items():
        cursor.execute(sql, (
            matchId,
            inningsId,
            wicket["wktNbr"],
            wicket["batId"],
            wicket["batName"],
            wicket["wktOver"],
            wicket["wktRuns"],
        ))
    conn.commit()

In [116]:
def insert_partnerships(cursor, partnerships_data, matchId, inningsId):
    # cursor = conn.cursor()
    sql = """INSERT INTO Partnerships (matchId, inningsId, partnershipNbr, bat1Id,bat1Name,bat1Runs,bat1balls, bat2Id,bat2Name,bat2Runs,bat2balls,totalRuns, totalBalls)
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    
    for i, (_, partnership) in enumerate(partnerships_data.items(), start=1):
        cursor.execute(sql, (
            matchId,
            inningsId,
            i,  # Assigns a sequential partnership number
            partnership["bat1Id"],
            partnership["bat1Name"],
            partnership["bat1Runs"],
            partnership["bat1balls"],
            partnership["bat2Id"],
            partnership["bat2Name"],
            partnership["bat2Runs"],
            partnership["bat2balls"],
            partnership["totalRuns"],
            partnership["totalBalls"]
        ))
    # conn.commit()

In [158]:
match_data = fetch_match_data()

In [ ]:
match_data

In [161]:
if match_data is not None:
    
    matchId = match_data["matchHeader"]["matchId"]
    
    # Insert Series and Match Data
    # insert_series(cs, match_data["matchHeader"])
    # insert_match(cs, match_data["matchHeader"])

    # # Insert Team Data
    # insert_team(cs, match_data["matchHeader"]["team1"])
    # insert_team(cs, match_data["matchHeader"]["team2"])

    # # Insert Player Data
    # insert_players(cs, match_data["matchHeader"]["team1"]["playerDetails"], match_data["matchHeader"]["team1"]["id"])
    # insert_players(cs, match_data["matchHeader"]["team2"]["playerDetails"], match_data["matchHeader"]["team2"]["id"])

    # # Insert Innings Data
    for innings in match_data["scoreCard"]:
        # insert_innings(cs, innings)
        # insert_batting_performance(cs, innings["batTeamDetails"]["batsmenData"],match_id, innings["inningsId"])
        # insert_bowling_performance(cs, innings["bowlTeamDetails"]["bowlersData"],match_id, innings["inningsId"])
        # insert_partnerships(cs, innings["partnershipsData"], matchId, innings["inningsId"])
        insert_wickets(cs, innings["wicketsData"], matchId, innings["inningsId"])

    # Close Connection
    # conn.close()